# Day 7: Bayesian Methods - Interview Questions & Week Summary

## Week 20 | Quant ML Learning Path

**Objective:** Master 10 essential Bayesian interview questions and consolidate week's learning

---

### Interview Topics Covered:
1. **Bayes' Theorem** - Foundation of Bayesian inference
2. **Prior vs Posterior** - Belief updating mechanism
3. **Conjugate Priors** - Analytical tractability
4. **MAP Estimation** - Point estimation with priors
5. **Bayesian vs Frequentist** - Philosophical and practical differences
6. **MCMC Methods** - Sampling from complex posteriors
7. **Bayesian Linear Regression** - Uncertainty quantification
8. **Naive Bayes Classifier** - Probabilistic classification
9. **Credible vs Confidence Intervals** - Interval interpretation
10. **Bayesian Model Selection** - Model comparison techniques

---

**Finance Applications:** Portfolio optimization under uncertainty, risk modeling, strategy parameter estimation, regime detection

## Import Required Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from scipy.special import gamma, beta as beta_func
from scipy.optimize import minimize
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

# Plotting style
plt.style.use('seaborn-v0_8-whitegrid')
np.random.seed(42)

print("✅ Libraries loaded successfully")
print("=" * 60)
print("Day 7: Bayesian Interview Questions & Week Summary")
print("=" * 60)

---

## Question 1: Explain Bayes' Theorem and Implement It

### 📋 Interview Question:
*"Explain Bayes' theorem and provide a practical example. How would you implement it to calculate the probability of a trading strategy being profitable given a positive signal?"*

### 🎯 Key Concepts:
- **Bayes' Theorem:** $P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B)}$
- **Components:** Prior P(A), Likelihood P(B|A), Evidence P(B), Posterior P(A|B)
- **Interpretation:** Update beliefs with new evidence

In [ ]:
def bayes_theorem(prior, likelihood, evidence):
    """
    Calculate posterior probability using Bayes' theorem.
    
    P(A|B) = P(B|A) * P(A) / P(B)
    
    Parameters:
    -----------
    prior : float - P(A) - Prior probability of hypothesis
    likelihood : float - P(B|A) - Probability of evidence given hypothesis
    evidence : float - P(B) - Total probability of evidence
    
    Returns:
    --------
    float - Posterior probability P(A|B)
    """
    posterior = (likelihood * prior) / evidence
    return posterior


# Example: Trading Strategy Signal Analysis
print("=" * 60)
print("BAYES' THEOREM: Trading Strategy Example")
print("=" * 60)

# Scenario: Signal predicts profitable trade
# Historical data:
# - P(Profitable) = 0.55 (55% of trades are profitable - prior)
# - P(Signal | Profitable) = 0.80 (signal appears 80% of time when profitable - likelihood)
# - P(Signal | Not Profitable) = 0.30 (false positive rate)

P_profitable = 0.55  # Prior
P_signal_given_profitable = 0.80  # Likelihood
P_signal_given_not_profitable = 0.30  # False positive rate

# Calculate P(Signal) using law of total probability
P_signal = (P_signal_given_profitable * P_profitable + 
            P_signal_given_not_profitable * (1 - P_profitable))

# Apply Bayes' theorem
P_profitable_given_signal = bayes_theorem(
    prior=P_profitable,
    likelihood=P_signal_given_profitable,
    evidence=P_signal
)

print(f"\n📊 Given Information:")
print(f"   P(Profitable) = {P_profitable:.2%} [Prior]")
print(f"   P(Signal | Profitable) = {P_signal_given_profitable:.2%} [Likelihood]")
print(f"   P(Signal | Not Profitable) = {P_signal_given_not_profitable:.2%} [False Positive]")
print(f"\n📐 Calculated:")
print(f"   P(Signal) = {P_signal:.2%} [Evidence]")
print(f"\n✅ P(Profitable | Signal) = {P_profitable_given_signal:.2%} [Posterior]")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Components of Bayes' Theorem
components = ['Prior\nP(A)', 'Likelihood\nP(B|A)', 'Evidence\nP(B)', 'Posterior\nP(A|B)']
values = [P_profitable, P_signal_given_profitable, P_signal, P_profitable_given_signal]
colors = ['#3498db', '#2ecc71', '#e74c3c', '#9b59b6']

bars = axes[0].bar(components, values, color=colors, edgecolor='black', linewidth=1.5)
axes[0].set_ylim(0, 1)
axes[0].set_ylabel('Probability', fontsize=12)
axes[0].set_title("Bayes' Theorem Components", fontsize=14, fontweight='bold')
for bar, val in zip(bars, values):
    axes[0].annotate(f'{val:.1%}', xy=(bar.get_x() + bar.get_width()/2, val),
                     ha='center', va='bottom', fontsize=11, fontweight='bold')

# Right: Posterior update visualization
categories = ['Before Signal\n(Prior)', 'After Signal\n(Posterior)']
prob_values = [P_profitable, P_profitable_given_signal]

bars2 = axes[1].bar(categories, prob_values, color=['#3498db', '#9b59b6'], 
                     edgecolor='black', linewidth=1.5, width=0.5)
axes[1].axhline(y=0.5, color='red', linestyle='--', label='Break-even', alpha=0.7)
axes[1].set_ylim(0, 1)
axes[1].set_ylabel('P(Profitable)', fontsize=12)
axes[1].set_title('Belief Update with Evidence', fontsize=14, fontweight='bold')
axes[1].legend()
for bar, val in zip(bars2, prob_values):
    axes[1].annotate(f'{val:.1%}', xy=(bar.get_x() + bar.get_width()/2, val),
                     ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\n💡 Interview Answer Key Points:")
print("   1. Bayes' theorem relates conditional probabilities")
print("   2. Allows updating prior beliefs with new evidence")
print("   3. Essential for sequential learning and signal processing")
print("   4. In finance: refine probability estimates as data arrives")

---

## Question 2: Prior vs Posterior Distribution

### 📋 Interview Question:
*"Explain the difference between prior and posterior distributions. Demonstrate how observing data updates our beliefs using a coin flip example."*

### 🎯 Key Concepts:
- **Prior:** Initial belief before seeing data
- **Posterior:** Updated belief after seeing data
- **Beta-Binomial Model:** Conjugate pair for binary outcomes

In [ ]:
print("=" * 60)
print("PRIOR VS POSTERIOR: Beta-Binomial Model")
print("=" * 60)

# Beta-Binomial conjugate pair for coin flips
# Prior: Beta(α, β) → models belief about probability θ
# Likelihood: Binomial(n, k | θ)
# Posterior: Beta(α + k, β + n - k)

class BetaBinomialModel:
    """Bayesian model for binary outcomes using Beta-Binomial conjugacy."""
    
    def __init__(self, alpha_prior=1, beta_prior=1):
        """
        Initialize with prior parameters.
        
        Parameters:
        -----------
        alpha_prior : float - Prior successes + 1 (pseudo-counts)
        beta_prior : float - Prior failures + 1 (pseudo-counts)
        
        Common priors:
        - Beta(1, 1): Uniform - no prior knowledge
        - Beta(0.5, 0.5): Jeffreys - non-informative
        - Beta(2, 2): Weakly informative - slight preference for θ=0.5
        """
        self.alpha_prior = alpha_prior
        self.beta_prior = beta_prior
        self.alpha_post = alpha_prior
        self.beta_post = beta_prior
        self.n_observations = 0
        self.n_successes = 0
        
    def update(self, successes, failures):
        """Update posterior with new observations."""
        self.n_successes += successes
        self.n_observations += successes + failures
        self.alpha_post = self.alpha_prior + self.n_successes
        self.beta_post = self.beta_prior + (self.n_observations - self.n_successes)
        
    def get_prior(self):
        """Return prior distribution."""
        return stats.beta(self.alpha_prior, self.beta_prior)
    
    def get_posterior(self):
        """Return posterior distribution."""
        return stats.beta(self.alpha_post, self.beta_post)
    
    def posterior_stats(self):
        """Calculate posterior summary statistics."""
        posterior = self.get_posterior()
        return {
            'mean': posterior.mean(),
            'std': posterior.std(),
            'mode': (self.alpha_post - 1) / (self.alpha_post + self.beta_post - 2) 
                    if self.alpha_post > 1 and self.beta_post > 1 else None,
            '95% CI': posterior.interval(0.95)
        }

# Simulate: Strategy win rate estimation
# True win rate = 0.65 (unknown to us)
true_win_rate = 0.65
np.random.seed(42)

# Generate trade outcomes
n_trades = 100
trade_outcomes = np.random.binomial(1, true_win_rate, n_trades)

# Start with uniform prior (no prior knowledge)
model = BetaBinomialModel(alpha_prior=1, beta_prior=1)

# Track posterior evolution
observation_points = [0, 10, 30, 100]
posteriors = {}
x = np.linspace(0, 1, 200)

# Store initial prior
posteriors[0] = {'dist': model.get_prior(), 'alpha': 1, 'beta': 1}

# Update sequentially
for i, outcome in enumerate(trade_outcomes):
    model.update(successes=outcome, failures=1-outcome)
    if i + 1 in observation_points:
        posteriors[i + 1] = {
            'dist': model.get_posterior(),
            'alpha': model.alpha_post,
            'beta': model.beta_post
        }

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

colors = ['#3498db', '#2ecc71', '#f39c12', '#e74c3c']
for ax, (n_obs, color) in zip(axes.flat, zip(observation_points, colors)):
    post = posteriors[n_obs]
    
    if n_obs == 0:
        label = f'Prior: Beta({post["alpha"]}, {post["beta"]})'
        title = 'Prior Distribution (n=0)'
    else:
        wins = sum(trade_outcomes[:n_obs])
        label = f'Posterior: Beta({post["alpha"]}, {post["beta"]})'
        title = f'After {n_obs} trades ({wins} wins)'
    
    y = post['dist'].pdf(x)
    ax.fill_between(x, y, alpha=0.3, color=color)
    ax.plot(x, y, color=color, linewidth=2, label=label)
    ax.axvline(x=true_win_rate, color='red', linestyle='--', 
               linewidth=2, label=f'True rate: {true_win_rate}')
    ax.axvline(x=post['dist'].mean(), color=color, linestyle=':',
               linewidth=2, label=f'Posterior mean: {post["dist"].mean():.3f}')
    
    ax.set_xlabel('Win Rate (θ)', fontsize=11)
    ax.set_ylabel('Density', fontsize=11)
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.legend(fontsize=9)
    ax.set_xlim(0, 1)

plt.suptitle('Bayesian Belief Update: Prior → Posterior', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# Final statistics
print(f"\n📊 Final Posterior Analysis (n={n_trades}):")
final_stats = model.posterior_stats()
print(f"   Posterior Mean: {final_stats['mean']:.4f}")
print(f"   Posterior Std: {final_stats['std']:.4f}")
print(f"   Posterior Mode: {final_stats['mode']:.4f}")
print(f"   95% Credible Interval: [{final_stats['95% CI'][0]:.4f}, {final_stats['95% CI'][1]:.4f}]")
print(f"   True Win Rate: {true_win_rate}")
print(f"   Sample Win Rate (MLE): {sum(trade_outcomes)/len(trade_outcomes):.4f}")

print(f"\n💡 Interview Answer Key Points:")
print("   1. Prior encodes beliefs BEFORE seeing data")
print("   2. Posterior is updated belief AFTER seeing data")
print("   3. More data → posterior concentrates around true value")
print("   4. Prior influence diminishes with more observations")

---

## Question 3: Conjugate Priors

### 📋 Interview Question:
*"What are conjugate priors and why are they useful? Provide examples of common conjugate pairs."*

### 🎯 Key Concepts:
- **Conjugate Prior:** Prior and posterior belong to same family
- **Analytical Tractability:** Closed-form posterior updates
- **Common Pairs:** Beta-Binomial, Normal-Normal, Gamma-Poisson

In [ ]:
print("=" * 60)
print("CONJUGATE PRIORS: Common Pairs in Finance")
print("=" * 60)

fig, axes = plt.subplots(1, 3, figsize=(16, 5))
x = np.linspace(0.001, 0.999, 200)

# ===== 1. Beta-Binomial (Binary outcomes) =====
# Use case: Win rate estimation, success probability
print("\n1️⃣ BETA-BINOMIAL CONJUGATE PAIR")
print("-" * 40)
print("   Likelihood: Binomial(n, k | θ)")
print("   Prior: Beta(α, β)")
print("   Posterior: Beta(α + k, β + n - k)")
print("   Use: Win rate, default probability, conversion rates")

alpha_prior, beta_prior = 2, 2  # Weakly informative prior
n_trials, successes = 50, 35

# Prior and Posterior
prior_beta = stats.beta(alpha_prior, beta_prior)
post_beta = stats.beta(alpha_prior + successes, beta_prior + n_trials - successes)

axes[0].fill_between(x, prior_beta.pdf(x), alpha=0.3, color='blue', label=f'Prior: Beta({alpha_prior},{beta_prior})')
axes[0].fill_between(x, post_beta.pdf(x), alpha=0.3, color='red', 
                      label=f'Posterior: Beta({alpha_prior+successes},{beta_prior+n_trials-successes})')
axes[0].plot(x, prior_beta.pdf(x), 'b-', linewidth=2)
axes[0].plot(x, post_beta.pdf(x), 'r-', linewidth=2)
axes[0].axvline(successes/n_trials, color='green', linestyle='--', label=f'MLE: {successes/n_trials:.2f}')
axes[0].set_xlabel('θ (Success Probability)', fontsize=11)
axes[0].set_ylabel('Density', fontsize=11)
axes[0].set_title('Beta-Binomial\n(Binary Outcomes)', fontsize=12, fontweight='bold')
axes[0].legend(fontsize=9)

# ===== 2. Normal-Normal (Mean estimation with known variance) =====
# Use case: Return estimation, parameter updates
print("\n2️⃣ NORMAL-NORMAL CONJUGATE PAIR")
print("-" * 40)
print("   Likelihood: Normal(μ, σ²) [σ known]")
print("   Prior: Normal(μ₀, σ₀²)")
print("   Posterior: Normal(μₙ, σₙ²)")
print("   Use: Return estimation, value estimation")

# Prior parameters
mu_prior, sigma_prior = 0.05, 0.10  # Prior: mean=5%, std=10%
sigma_known = 0.20  # Known observation noise

# Observed data
data = np.array([0.08, 0.12, -0.02, 0.15, 0.06, 0.10, 0.03])
n = len(data)
x_bar = np.mean(data)

# Posterior calculation
precision_prior = 1 / sigma_prior**2
precision_data = n / sigma_known**2
precision_post = precision_prior + precision_data

mu_post = (precision_prior * mu_prior + precision_data * x_bar) / precision_post
sigma_post = np.sqrt(1 / precision_post)

x_norm = np.linspace(-0.3, 0.4, 200)
prior_norm = stats.norm(mu_prior, sigma_prior)
post_norm = stats.norm(mu_post, sigma_post)

axes[1].fill_between(x_norm, prior_norm.pdf(x_norm), alpha=0.3, color='blue', 
                      label=f'Prior: N({mu_prior:.2f}, {sigma_prior:.2f}²)')
axes[1].fill_between(x_norm, post_norm.pdf(x_norm), alpha=0.3, color='red',
                      label=f'Posterior: N({mu_post:.2f}, {sigma_post:.2f}²)')
axes[1].plot(x_norm, prior_norm.pdf(x_norm), 'b-', linewidth=2)
axes[1].plot(x_norm, post_norm.pdf(x_norm), 'r-', linewidth=2)
axes[1].axvline(x_bar, color='green', linestyle='--', label=f'Sample Mean: {x_bar:.2f}')
axes[1].set_xlabel('μ (Mean Return)', fontsize=11)
axes[1].set_ylabel('Density', fontsize=11)
axes[1].set_title('Normal-Normal\n(Mean Estimation)', fontsize=12, fontweight='bold')
axes[1].legend(fontsize=9)

# ===== 3. Gamma-Poisson (Count data) =====
# Use case: Trade arrival rate, event frequency
print("\n3️⃣ GAMMA-POISSON CONJUGATE PAIR")
print("-" * 40)
print("   Likelihood: Poisson(k | λ)")
print("   Prior: Gamma(α, β)")
print("   Posterior: Gamma(α + Σk, β + n)")
print("   Use: Trade frequency, event counts, order arrivals")

alpha_gamma, beta_gamma = 3, 1  # Prior
observed_counts = np.array([5, 8, 6, 7, 9, 4, 6])  # Daily trade counts
n_obs = len(observed_counts)
sum_counts = np.sum(observed_counts)

# Posterior
alpha_post = alpha_gamma + sum_counts
beta_post = beta_gamma + n_obs

x_gamma = np.linspace(0.01, 15, 200)
prior_gamma = stats.gamma(alpha_gamma, scale=1/beta_gamma)
post_gamma = stats.gamma(alpha_post, scale=1/beta_post)

axes[2].fill_between(x_gamma, prior_gamma.pdf(x_gamma), alpha=0.3, color='blue',
                      label=f'Prior: Gamma({alpha_gamma}, {beta_gamma})')
axes[2].fill_between(x_gamma, post_gamma.pdf(x_gamma), alpha=0.3, color='red',
                      label=f'Posterior: Gamma({alpha_post}, {beta_post})')
axes[2].plot(x_gamma, prior_gamma.pdf(x_gamma), 'b-', linewidth=2)
axes[2].plot(x_gamma, post_gamma.pdf(x_gamma), 'r-', linewidth=2)
axes[2].axvline(np.mean(observed_counts), color='green', linestyle='--', 
                label=f'MLE: {np.mean(observed_counts):.2f}')
axes[2].set_xlabel('λ (Rate Parameter)', fontsize=11)
axes[2].set_ylabel('Density', fontsize=11)
axes[2].set_title('Gamma-Poisson\n(Count Data)', fontsize=12, fontweight='bold')
axes[2].legend(fontsize=9)

plt.suptitle('Conjugate Prior Pairs: Prior (Blue) → Posterior (Red)', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# Summary table
print("\n" + "=" * 60)
print("📚 CONJUGATE PRIOR REFERENCE TABLE")
print("=" * 60)
conjugate_table = """
┌─────────────────┬─────────────────┬─────────────────┬──────────────────────┐
│ Likelihood      │ Prior           │ Posterior       │ Finance Application  │
├─────────────────┼─────────────────┼─────────────────┼──────────────────────┤
│ Binomial        │ Beta            │ Beta            │ Win rate, hit ratio  │
│ Normal (μ)      │ Normal          │ Normal          │ Expected returns     │
│ Normal (σ²)     │ Inv-Gamma       │ Inv-Gamma       │ Volatility           │
│ Poisson         │ Gamma           │ Gamma           │ Trade frequency      │
│ Exponential     │ Gamma           │ Gamma           │ Time between trades  │
│ Multinomial     │ Dirichlet       │ Dirichlet       │ Regime probabilities │
└─────────────────┴─────────────────┴─────────────────┴──────────────────────┘
"""
print(conjugate_table)

print(f"\n💡 Interview Answer Key Points:")
print("   1. Conjugacy = prior & posterior same distribution family")
print("   2. Enables closed-form posterior (no MCMC needed)")
print("   3. Computationally efficient for real-time updates")
print("   4. Prior hyperparameters = pseudo-observations")

---

## Question 4: Maximum A Posteriori (MAP) Estimation

### 📋 Interview Question:
*"What is MAP estimation and how does it differ from MLE? When would you prefer one over the other?"*

### 🎯 Key Concepts:
- **MLE:** $\hat{\theta}_{MLE} = \arg\max_\theta P(D|\theta)$
- **MAP:** $\hat{\theta}_{MAP} = \arg\max_\theta P(\theta|D) = \arg\max_\theta P(D|\theta)P(\theta)$
- **Regularization:** MAP with Gaussian prior = L2 regularization

In [ ]:
print("=" * 60)
print("MAP VS MLE: Point Estimation Comparison")
print("=" * 60)

# Generate data with small sample size to show MAP benefits
np.random.seed(42)
true_theta = 0.3  # True success probability (rare event)
n_samples = 10
data = np.random.binomial(1, true_theta, n_samples)
successes = sum(data)
failures = n_samples - successes

print(f"\n📊 Scenario: Estimating rare event probability")
print(f"   True probability: {true_theta}")
print(f"   Observed: {successes} successes out of {n_samples} trials")

# MLE Estimation
theta_mle = successes / n_samples
print(f"\n1️⃣ MLE Estimate: {theta_mle:.4f}")
print(f"   Formula: k/n = {successes}/{n_samples}")

# MAP Estimation with Beta prior
def neg_log_posterior(theta, alpha, beta, k, n):
    """Negative log posterior for Beta-Binomial model."""
    if theta <= 0 or theta >= 1:
        return np.inf
    log_likelihood = k * np.log(theta) + (n - k) * np.log(1 - theta)
    log_prior = (alpha - 1) * np.log(theta) + (beta - 1) * np.log(1 - theta)
    return -(log_likelihood + log_prior)

# Try different priors
priors = {
    'Uniform Beta(1,1)': (1, 1),
    'Informative Beta(3,7)': (3, 7),  # Prior belief: ~30% success
    'Skeptical Beta(2,8)': (2, 8),    # Prior belief: ~20% success
}

print(f"\n2️⃣ MAP Estimates with Different Priors:")
map_estimates = {}

for name, (alpha, beta) in priors.items():
    # Analytical MAP for Beta-Binomial
    theta_map = (alpha + successes - 1) / (alpha + beta + n_samples - 2)
    map_estimates[name] = theta_map
    print(f"   {name}: θ_MAP = {theta_map:.4f}")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Different estimators comparison
theta_range = np.linspace(0.001, 0.999, 200)

# Likelihood function
log_likelihood = successes * np.log(theta_range) + failures * np.log(1 - theta_range)
likelihood = np.exp(log_likelihood - np.max(log_likelihood))
likelihood /= np.max(likelihood)

axes[0].plot(theta_range, likelihood, 'k-', linewidth=2, label='Likelihood')
axes[0].axvline(theta_mle, color='blue', linestyle='--', linewidth=2, label=f'MLE: {theta_mle:.3f}')
axes[0].axvline(true_theta, color='green', linestyle='-', linewidth=2, label=f'True: {true_theta:.3f}')

colors = ['red', 'orange', 'purple']
for (name, theta_map), color in zip(map_estimates.items(), colors):
    axes[0].axvline(theta_map, color=color, linestyle=':', linewidth=2, label=f'MAP {name.split()[0]}: {theta_map:.3f}')

axes[0].set_xlabel('θ', fontsize=12)
axes[0].set_ylabel('Normalized Likelihood', fontsize=12)
axes[0].set_title('MLE vs MAP Estimates', fontsize=12, fontweight='bold')
axes[0].legend(fontsize=8, loc='upper right')
axes[0].set_xlim(0, 1)

# Right: MAP as regularization in regression
# Ridge regression = MAP with Gaussian prior
np.random.seed(42)
n_points = 20
X = np.random.randn(n_points, 5)  # 5 features, small dataset
true_weights = np.array([1.0, -0.5, 0.2, 0, 0])  # Sparse true weights
noise = np.random.randn(n_points) * 0.5
y = X @ true_weights + noise

# OLS (MLE)
w_ols = np.linalg.lstsq(X, y, rcond=None)[0]

# Ridge (MAP with Gaussian prior)
from sklearn.linear_model import Ridge
lambdas = [0.01, 0.1, 1.0, 10.0]
ridge_weights = {}
for lam in lambdas:
    ridge = Ridge(alpha=lam, fit_intercept=False)
    ridge.fit(X, y)
    ridge_weights[lam] = ridge.coef_

# Plot weight comparison
x_pos = np.arange(5)
width = 0.15

axes[1].bar(x_pos - 2*width, true_weights, width, label='True Weights', color='green', alpha=0.8)
axes[1].bar(x_pos - width, w_ols, width, label='OLS (MLE)', color='blue', alpha=0.8)
for i, lam in enumerate([0.1, 1.0]):
    axes[1].bar(x_pos + i*width, ridge_weights[lam], width, 
                label=f'Ridge λ={lam} (MAP)', alpha=0.8)

axes[1].set_xlabel('Feature Index', fontsize=12)
axes[1].set_ylabel('Weight Value', fontsize=12)
axes[1].set_title('MAP = Regularization in Regression', fontsize=12, fontweight='bold')
axes[1].set_xticks(x_pos)
axes[1].legend(fontsize=9)
axes[1].axhline(0, color='gray', linestyle='-', linewidth=0.5)

plt.tight_layout()
plt.show()

# Summary
print("\n" + "=" * 60)
print("📚 MAP VS MLE COMPARISON")
print("=" * 60)
comparison_table = """
┌────────────────────┬─────────────────────────┬─────────────────────────┐
│ Aspect             │ MLE                     │ MAP                     │
├────────────────────┼─────────────────────────┼─────────────────────────┤
│ Formula            │ argmax P(D|θ)           │ argmax P(θ|D)           │
│ Prior              │ None (implicit uniform) │ Explicit prior P(θ)     │
│ Small samples      │ Overfits                │ Regularized estimate    │
│ Large samples      │ Consistent              │ Converges to MLE        │
│ Equivalent to      │ No regularization       │ L2 regularization       │
│ Use case           │ Abundant data           │ Limited data + beliefs  │
└────────────────────┴─────────────────────────┴─────────────────────────┘
"""
print(comparison_table)

print(f"\n💡 Interview Answer Key Points:")
print("   1. MLE maximizes likelihood; MAP maximizes posterior")
print("   2. MAP includes prior beliefs, acts as regularization")
print("   3. Gaussian prior on weights → L2 (Ridge) regularization")
print("   4. Laplace prior on weights → L1 (Lasso) regularization")
print("   5. With infinite data, MAP → MLE (prior becomes irrelevant)")

---

## Question 5: Bayesian vs Frequentist Approach

### 📋 Interview Question:
*"Compare and contrast Bayesian and Frequentist approaches to statistics. When would you use each in quantitative finance?"*

### 🎯 Key Concepts:
- **Philosophical Difference:** Probability as degree of belief vs long-run frequency
- **Parameters:** Random (Bayesian) vs fixed but unknown (Frequentist)
- **Inference:** Posterior distribution vs point estimates + confidence intervals

In [ ]:
print("=" * 60)
print("BAYESIAN VS FREQUENTIST: Side-by-Side Comparison")
print("=" * 60)

# Scenario: Estimate expected return of a strategy
np.random.seed(42)
true_mu = 0.08  # 8% annual return
true_sigma = 0.20  # 20% volatility
n_observations = 30  # Monthly returns for 2.5 years

# Generate monthly returns
monthly_returns = np.random.normal(true_mu/12, true_sigma/np.sqrt(12), n_observations)
annual_return_est = np.mean(monthly_returns) * 12
annual_vol_est = np.std(monthly_returns) * np.sqrt(12)

print(f"\n📊 Scenario: Estimate Strategy's Expected Annual Return")
print(f"   True μ = {true_mu:.1%}, True σ = {true_sigma:.1%}")
print(f"   Sample: {n_observations} monthly returns")
print(f"   Sample mean (annualized): {annual_return_est:.2%}")

# ============== FREQUENTIST APPROACH ==============
print("\n" + "=" * 50)
print("🔵 FREQUENTIST APPROACH")
print("=" * 50)

# Point estimate
mu_freq = np.mean(monthly_returns) * 12

# Standard error
se_freq = (np.std(monthly_returns, ddof=1) / np.sqrt(n_observations)) * 12

# 95% Confidence Interval
from scipy.stats import t as t_dist
t_critical = t_dist.ppf(0.975, df=n_observations-1)
ci_freq = (mu_freq - t_critical * se_freq, mu_freq + t_critical * se_freq)

print(f"   Point Estimate: μ = {mu_freq:.2%}")
print(f"   Standard Error: {se_freq:.2%}")
print(f"   95% Confidence Interval: [{ci_freq[0]:.2%}, {ci_freq[1]:.2%}]")
print(f"\n   ⚠️ Interpretation: If we repeated this experiment many times,")
print(f"      95% of intervals would contain the true parameter.")
print(f"      We CANNOT say there's a 95% probability μ is in this interval.")

# ============== BAYESIAN APPROACH ==============
print("\n" + "=" * 50)
print("🟢 BAYESIAN APPROACH")
print("=" * 50)

# Prior: Informed by historical market data
# Assume historical equity premium ~6-8% with uncertainty
mu_prior = 0.07  # 7% prior expected return
sigma_prior = 0.05  # Prior uncertainty on mean

# Known volatility (simplification)
sigma_known = true_sigma

# Bayesian update (Normal-Normal conjugate)
precision_prior = 1 / sigma_prior**2
precision_data = n_observations / (sigma_known/np.sqrt(12))**2  # Monthly data precision

# Posterior parameters
precision_post = precision_prior + precision_data
mu_post = (precision_prior * mu_prior + precision_data * annual_return_est) / precision_post
sigma_post = np.sqrt(1 / precision_post)

# 95% Credible Interval
ci_bayes = stats.norm(mu_post, sigma_post).interval(0.95)

print(f"   Prior: μ ~ N({mu_prior:.1%}, {sigma_prior:.1%}²)")
print(f"   Posterior: μ ~ N({mu_post:.2%}, {sigma_post:.2%}²)")
print(f"   95% Credible Interval: [{ci_bayes[0]:.2%}, {ci_bayes[1]:.2%}]")
print(f"\n   ✅ Interpretation: Given the data and prior,")
print(f"      there is a 95% probability μ lies in this interval.")

# Additional Bayesian insights
prob_positive = 1 - stats.norm(mu_post, sigma_post).cdf(0)
prob_beat_rf = 1 - stats.norm(mu_post, sigma_post).cdf(0.04)  # Beat 4% risk-free
print(f"\n   P(μ > 0) = {prob_positive:.1%}")
print(f"   P(μ > 4% risk-free) = {prob_beat_rf:.1%}")

# Visualization
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Plot 1: Frequentist CI interpretation
ax = axes[0]
np.random.seed(123)
n_simulations = 20
cis = []
contains_true = []

for i in range(n_simulations):
    sample = np.random.normal(true_mu/12, true_sigma/np.sqrt(12), n_observations)
    mean_i = np.mean(sample) * 12
    se_i = (np.std(sample, ddof=1) / np.sqrt(n_observations)) * 12
    ci_i = (mean_i - t_critical * se_i, mean_i + t_critical * se_i)
    cis.append((mean_i, ci_i))
    contains_true.append(ci_i[0] <= true_mu <= ci_i[1])

for i, (mean_i, ci_i) in enumerate(cis):
    color = 'green' if contains_true[i] else 'red'
    ax.plot([ci_i[0], ci_i[1]], [i, i], color=color, linewidth=2)
    ax.scatter([mean_i], [i], color=color, s=30, zorder=5)

ax.axvline(true_mu, color='blue', linestyle='--', linewidth=2, label=f'True μ = {true_mu:.1%}')
ax.set_xlabel('Estimated Return', fontsize=11)
ax.set_ylabel('Simulation', fontsize=11)
ax.set_title(f'Frequentist: 95% CI Coverage\n{sum(contains_true)}/{n_simulations} contain true μ', 
             fontsize=12, fontweight='bold')
ax.legend()

# Plot 2: Bayesian posterior with prior
ax = axes[1]
x = np.linspace(-0.1, 0.25, 200)

prior_dist = stats.norm(mu_prior, sigma_prior)
post_dist = stats.norm(mu_post, sigma_post)

ax.fill_between(x, prior_dist.pdf(x), alpha=0.3, color='blue', label=f'Prior N({mu_prior:.0%}, {sigma_prior:.0%}²)')
ax.fill_between(x, post_dist.pdf(x), alpha=0.3, color='green', label=f'Posterior N({mu_post:.1%}, {sigma_post:.1%}²)')
ax.plot(x, prior_dist.pdf(x), 'b-', linewidth=2)
ax.plot(x, post_dist.pdf(x), 'g-', linewidth=2)
ax.axvline(true_mu, color='red', linestyle='--', linewidth=2, label=f'True μ = {true_mu:.0%}')
ax.axvline(mu_post, color='green', linestyle=':', linewidth=2, label=f'Posterior mean')
ax.axvspan(ci_bayes[0], ci_bayes[1], alpha=0.2, color='green', label='95% Credible Interval')

ax.set_xlabel('Expected Return (μ)', fontsize=11)
ax.set_ylabel('Density', fontsize=11)
ax.set_title('Bayesian: Prior → Posterior Update', fontsize=12, fontweight='bold')
ax.legend(fontsize=9)

# Plot 3: Comparison summary
ax = axes[2]
comparison_data = {
    'Parameters': ['Point Est.', '95% Lower', '95% Upper', 'Width'],
    'Frequentist': [mu_freq, ci_freq[0], ci_freq[1], ci_freq[1]-ci_freq[0]],
    'Bayesian': [mu_post, ci_bayes[0], ci_bayes[1], ci_bayes[1]-ci_bayes[0]],
}

x_pos = np.arange(4)
width = 0.35

bars1 = ax.bar(x_pos - width/2, comparison_data['Frequentist'], width, label='Frequentist', color='blue', alpha=0.7)
bars2 = ax.bar(x_pos + width/2, comparison_data['Bayesian'], width, label='Bayesian', color='green', alpha=0.7)

ax.axhline(true_mu, color='red', linestyle='--', linewidth=2, label=f'True μ = {true_mu:.1%}')
ax.set_ylabel('Value', fontsize=11)
ax.set_xticks(x_pos)
ax.set_xticklabels(comparison_data['Parameters'])
ax.set_title('Method Comparison', fontsize=12, fontweight='bold')
ax.legend(fontsize=9)

plt.tight_layout()
plt.show()

# Comparison Table
print("\n" + "=" * 60)
print("📚 BAYESIAN VS FREQUENTIST COMPARISON")
print("=" * 60)
print("""
┌─────────────────────┬─────────────────────────┬─────────────────────────┐
│ Aspect              │ Frequentist             │ Bayesian                │
├─────────────────────┼─────────────────────────┼─────────────────────────┤
│ Parameters          │ Fixed, unknown          │ Random variables        │
│ Probability         │ Long-run frequency      │ Degree of belief        │
│ Prior knowledge     │ Not used                │ Incorporated via prior  │
│ Uncertainty         │ Confidence intervals    │ Posterior distribution  │
│ Interpretation      │ Repeated sampling       │ Direct probability      │
│ Computation         │ Often analytical        │ May need MCMC           │
│ Small samples       │ Unreliable              │ Prior provides info     │
│ Large samples       │ Both converge           │ Both converge           │
└─────────────────────┴─────────────────────────┴─────────────────────────┘
""")

print(f"\n💡 Interview Answer Key Points:")
print("   1. Frequentist: parameters fixed; probability = long-run frequency")
print("   2. Bayesian: parameters random; probability = degree of belief")  
print("   3. Bayesian naturally incorporates prior knowledge")
print("   4. Credible intervals have direct probabilistic interpretation")
print("   5. Finance: Bayesian better for rare events, regime changes, small samples")

---

## Question 6: Markov Chain Monte Carlo (MCMC)

### 📋 Interview Question:
*"Explain MCMC and the Metropolis-Hastings algorithm. Why is it used in Bayesian inference?"*

### 🎯 Key Concepts:
- **Purpose:** Sample from complex posterior distributions
- **Markov Chain:** Sequence where next state depends only on current state
- **Metropolis-Hastings:** Accept/reject proposals based on posterior ratio

In [ ]:
print("=" * 60)
print("MCMC: Metropolis-Hastings Algorithm from Scratch")
print("=" * 60)

def metropolis_hastings(log_target, initial_state, proposal_std, n_samples, burn_in=1000):
    """
    Metropolis-Hastings MCMC sampler.
    
    Parameters:
    -----------
    log_target : callable - Log of target distribution (unnormalized posterior)
    initial_state : float or array - Starting point
    proposal_std : float - Standard deviation of Gaussian proposal
    n_samples : int - Number of samples to draw
    burn_in : int - Samples to discard at beginning
    
    Returns:
    --------
    samples : array - MCMC samples from target distribution
    acceptance_rate : float - Proportion of accepted proposals
    """
    samples = []
    current = initial_state
    current_log_prob = log_target(current)
    n_accepted = 0
    
    for i in range(n_samples + burn_in):
        # Propose new state (symmetric Gaussian proposal)
        proposal = current + np.random.normal(0, proposal_std)
        proposal_log_prob = log_target(proposal)
        
        # Calculate acceptance probability
        log_alpha = proposal_log_prob - current_log_prob
        
        # Accept or reject
        if np.log(np.random.random()) < log_alpha:
            current = proposal
            current_log_prob = proposal_log_prob
            if i >= burn_in:
                n_accepted += 1
        
        if i >= burn_in:
            samples.append(current)
    
    acceptance_rate = n_accepted / n_samples
    return np.array(samples), acceptance_rate


# Example: Estimate mean of returns with unknown mean AND variance
# Target posterior: p(μ, σ | data) ∝ p(data | μ, σ) * p(μ) * p(σ)

np.random.seed(42)
true_mu = 0.10
true_sigma = 0.25
data = np.random.normal(true_mu, true_sigma, 50)

print(f"\n📊 Problem: Estimate μ and σ from 50 return observations")
print(f"   True μ = {true_mu}, True σ = {true_sigma}")
print(f"   Sample mean = {np.mean(data):.4f}, Sample std = {np.std(data):.4f}")

# Define log posterior (unnormalized)
def log_posterior(params):
    """Log posterior for Normal model: p(μ, σ | data)"""
    mu, log_sigma = params
    sigma = np.exp(log_sigma)  # Transform to ensure σ > 0
    
    if sigma <= 0:
        return -np.inf
    
    # Log likelihood: Normal(data | μ, σ)
    log_likelihood = np.sum(stats.norm.logpdf(data, mu, sigma))
    
    # Priors: μ ~ N(0, 1), σ ~ Exp(1) [via log_σ ~ Logistic]
    log_prior_mu = stats.norm.logpdf(mu, 0, 1)
    log_prior_sigma = stats.expon.logpdf(sigma, scale=1) + log_sigma  # Jacobian
    
    return log_likelihood + log_prior_mu + log_prior_sigma

# Run MCMC
n_samples = 10000
burn_in = 2000
initial = np.array([0.0, np.log(0.3)])
proposal_std = 0.05

print(f"\n🔄 Running Metropolis-Hastings...")
print(f"   Samples: {n_samples}, Burn-in: {burn_in}")

# Run for both parameters together
samples_list = []
current = initial.copy()
current_log_prob = log_posterior(current)
n_accepted = 0

for i in range(n_samples + burn_in):
    proposal = current + np.random.normal(0, proposal_std, size=2)
    proposal_log_prob = log_posterior(proposal)
    
    log_alpha = proposal_log_prob - current_log_prob
    
    if np.log(np.random.random()) < log_alpha:
        current = proposal
        current_log_prob = proposal_log_prob
        if i >= burn_in:
            n_accepted += 1
    
    if i >= burn_in:
        samples_list.append(current.copy())

samples = np.array(samples_list)
mu_samples = samples[:, 0]
sigma_samples = np.exp(samples[:, 1])
acceptance_rate = n_accepted / n_samples

print(f"   Acceptance rate: {acceptance_rate:.1%}")

# Results
print(f"\n📈 Posterior Estimates:")
print(f"   μ: mean = {np.mean(mu_samples):.4f}, std = {np.std(mu_samples):.4f}")
print(f"      95% CI: [{np.percentile(mu_samples, 2.5):.4f}, {np.percentile(mu_samples, 97.5):.4f}]")
print(f"   σ: mean = {np.mean(sigma_samples):.4f}, std = {np.std(sigma_samples):.4f}")
print(f"      95% CI: [{np.percentile(sigma_samples, 2.5):.4f}, {np.percentile(sigma_samples, 97.5):.4f}]")

# Visualization
fig, axes = plt.subplots(2, 3, figsize=(15, 8))

# Row 1: μ analysis
# Trace plot
axes[0, 0].plot(mu_samples, alpha=0.7, linewidth=0.5)
axes[0, 0].axhline(true_mu, color='red', linestyle='--', linewidth=2, label=f'True μ = {true_mu}')
axes[0, 0].set_xlabel('Iteration', fontsize=11)
axes[0, 0].set_ylabel('μ', fontsize=11)
axes[0, 0].set_title('Trace Plot (μ)', fontsize=12, fontweight='bold')
axes[0, 0].legend()

# Histogram
axes[0, 1].hist(mu_samples, bins=50, density=True, alpha=0.7, color='blue', edgecolor='black')
axes[0, 1].axvline(true_mu, color='red', linestyle='--', linewidth=2, label=f'True μ = {true_mu}')
axes[0, 1].axvline(np.mean(mu_samples), color='green', linestyle=':', linewidth=2, 
                   label=f'Post. mean = {np.mean(mu_samples):.3f}')
axes[0, 1].set_xlabel('μ', fontsize=11)
axes[0, 1].set_ylabel('Density', fontsize=11)
axes[0, 1].set_title('Posterior Distribution (μ)', fontsize=12, fontweight='bold')
axes[0, 1].legend()

# Autocorrelation
max_lag = 100
autocorr = [np.corrcoef(mu_samples[:-lag], mu_samples[lag:])[0, 1] for lag in range(1, max_lag+1)]
axes[0, 2].bar(range(1, max_lag+1), autocorr, alpha=0.7)
axes[0, 2].axhline(0, color='black', linewidth=0.5)
axes[0, 2].set_xlabel('Lag', fontsize=11)
axes[0, 2].set_ylabel('Autocorrelation', fontsize=11)
axes[0, 2].set_title('Autocorrelation (μ)', fontsize=12, fontweight='bold')

# Row 2: σ analysis
axes[1, 0].plot(sigma_samples, alpha=0.7, linewidth=0.5, color='orange')
axes[1, 0].axhline(true_sigma, color='red', linestyle='--', linewidth=2, label=f'True σ = {true_sigma}')
axes[1, 0].set_xlabel('Iteration', fontsize=11)
axes[1, 0].set_ylabel('σ', fontsize=11)
axes[1, 0].set_title('Trace Plot (σ)', fontsize=12, fontweight='bold')
axes[1, 0].legend()

axes[1, 1].hist(sigma_samples, bins=50, density=True, alpha=0.7, color='orange', edgecolor='black')
axes[1, 1].axvline(true_sigma, color='red', linestyle='--', linewidth=2, label=f'True σ = {true_sigma}')
axes[1, 1].axvline(np.mean(sigma_samples), color='green', linestyle=':', linewidth=2,
                   label=f'Post. mean = {np.mean(sigma_samples):.3f}')
axes[1, 1].set_xlabel('σ', fontsize=11)
axes[1, 1].set_ylabel('Density', fontsize=11)
axes[1, 1].set_title('Posterior Distribution (σ)', fontsize=12, fontweight='bold')
axes[1, 1].legend()

# Joint posterior
axes[1, 2].scatter(mu_samples[::10], sigma_samples[::10], alpha=0.3, s=10)
axes[1, 2].scatter([true_mu], [true_sigma], color='red', s=100, marker='*', 
                   label=f'True (μ={true_mu}, σ={true_sigma})', zorder=5)
axes[1, 2].scatter([np.mean(mu_samples)], [np.mean(sigma_samples)], color='green', s=100, 
                   marker='x', linewidths=3, label='Posterior mean', zorder=5)
axes[1, 2].set_xlabel('μ', fontsize=11)
axes[1, 2].set_ylabel('σ', fontsize=11)
axes[1, 2].set_title('Joint Posterior', fontsize=12, fontweight='bold')
axes[1, 2].legend()

plt.tight_layout()
plt.show()

print(f"\n💡 Interview Answer Key Points:")
print("   1. MCMC samples from complex posteriors that can't be solved analytically")
print("   2. Metropolis-Hastings: propose → accept/reject based on posterior ratio")
print("   3. Burn-in discards initial samples before chain converges")
print("   4. Check convergence: trace plots, autocorrelation, R-hat statistic")
print("   5. Modern alternatives: Hamiltonian MC (HMC), No-U-Turn Sampler (NUTS)")

---

## Question 7: Bayesian Linear Regression

### 📋 Interview Question:
*"Explain Bayesian Linear Regression. How does it differ from OLS, and what are its advantages?"*

### 🎯 Key Concepts:
- **Weights as Distributions:** $P(w|D)$ instead of point estimates
- **Predictive Uncertainty:** Quantify confidence in predictions
- **Natural Regularization:** Prior acts as regularizer

In [ ]:
print("=" * 60)
print("BAYESIAN LINEAR REGRESSION: From Scratch")
print("=" * 60)

class BayesianLinearRegression:
    """
    Bayesian Linear Regression with conjugate Normal-InverseGamma prior.
    
    Model: y = Xw + ε, where ε ~ N(0, σ²)
    Prior: w ~ N(m₀, S₀), 1/σ² ~ Gamma(a₀, b₀)
    
    For known σ²:
    Posterior: w | D ~ N(mₙ, Sₙ)
    - Sₙ = (S₀⁻¹ + σ⁻²X'X)⁻¹
    - mₙ = Sₙ(S₀⁻¹m₀ + σ⁻²X'y)
    """
    
    def __init__(self, alpha=1.0, beta=1.0):
        """
        Parameters:
        -----------
        alpha : float - Precision of prior on weights (1/σ²_w)
        beta : float - Precision of noise (1/σ²_n)
        """
        self.alpha = alpha  # Prior precision
        self.beta = beta    # Noise precision
        self.m_n = None     # Posterior mean
        self.S_n = None     # Posterior covariance
        
    def fit(self, X, y):
        """Fit Bayesian linear regression."""
        n_samples, n_features = X.shape
        
        # Prior: w ~ N(0, α⁻¹I)
        S_0_inv = self.alpha * np.eye(n_features)
        m_0 = np.zeros(n_features)
        
        # Posterior covariance
        self.S_n = np.linalg.inv(S_0_inv + self.beta * X.T @ X)
        
        # Posterior mean
        self.m_n = self.S_n @ (S_0_inv @ m_0 + self.beta * X.T @ y)
        
        return self
    
    def predict(self, X, return_std=False):
        """
        Make predictions with uncertainty.
        
        Predictive distribution: p(y*|x*, D) = N(m_n'x*, σ²_pred)
        σ²_pred = 1/β + x*'Sₙx*
        """
        y_mean = X @ self.m_n
        
        if return_std:
            # Predictive variance = noise variance + weight uncertainty
            y_var = 1/self.beta + np.sum(X @ self.S_n * X, axis=1)
            return y_mean, np.sqrt(y_var)
        
        return y_mean
    
    def sample_weights(self, n_samples=100):
        """Sample weights from posterior."""
        return np.random.multivariate_normal(self.m_n, self.S_n, n_samples)


# Generate synthetic factor return prediction data
np.random.seed(42)
n_points = 50

# Features: factor exposures
X_raw = np.random.randn(n_points, 1)
X = np.hstack([np.ones((n_points, 1)), X_raw])  # Add intercept

# True relationship
true_w = np.array([0.02, 0.5])  # alpha=2%, beta=0.5
noise_std = 0.15

y = X @ true_w + np.random.randn(n_points) * noise_std

print(f"\n📊 Factor Return Prediction Problem")
print(f"   True weights: intercept={true_w[0]:.2f}, slope={true_w[1]:.2f}")
print(f"   Noise std: {noise_std}")
print(f"   Training samples: {n_points}")

# Fit models
# 1. OLS
ols = LinearRegression(fit_intercept=False)
ols.fit(X, y)

# 2. Bayesian Linear Regression
blr = BayesianLinearRegression(alpha=1.0, beta=1/noise_std**2)
blr.fit(X, y)

# 3. Sklearn BayesianRidge
sklearn_blr = BayesianRidge()
sklearn_blr.fit(X_raw, y)

print(f"\n📈 Weight Estimates:")
print(f"   OLS:      intercept={ols.coef_[0]:.4f}, slope={ols.coef_[1]:.4f}")
print(f"   Bayesian: intercept={blr.m_n[0]:.4f}, slope={blr.m_n[1]:.4f}")
print(f"   Weight uncertainty (posterior std):")
print(f"            intercept_std={np.sqrt(blr.S_n[0,0]):.4f}, slope_std={np.sqrt(blr.S_n[1,1]):.4f}")

# Predictions with uncertainty
X_test = np.linspace(-3, 3, 100).reshape(-1, 1)
X_test_full = np.hstack([np.ones((100, 1)), X_test])

y_ols = X_test_full @ ols.coef_
y_bayes, y_std = blr.predict(X_test_full, return_std=True)

# Visualization
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Plot 1: Data and predictions
ax = axes[0]
ax.scatter(X_raw, y, c='blue', alpha=0.6, label='Training data', s=40)
ax.plot(X_test, y_ols, 'g-', linewidth=2, label='OLS fit')
ax.plot(X_test, y_bayes, 'r-', linewidth=2, label='Bayesian mean')
ax.fill_between(X_test.ravel(), 
                y_bayes - 2*y_std, 
                y_bayes + 2*y_std, 
                color='red', alpha=0.2, label='95% predictive interval')
ax.plot(X_test, X_test_full @ true_w, 'k--', linewidth=2, label='True function')
ax.set_xlabel('Factor Exposure', fontsize=11)
ax.set_ylabel('Return', fontsize=11)
ax.set_title('OLS vs Bayesian Regression', fontsize=12, fontweight='bold')
ax.legend(fontsize=9)

# Plot 2: Posterior over weights (sample regression lines)
ax = axes[1]
ax.scatter(X_raw, y, c='blue', alpha=0.6, s=40)

# Sample from weight posterior
weight_samples = blr.sample_weights(n_samples=50)
for w_sample in weight_samples:
    y_sample = X_test_full @ w_sample
    ax.plot(X_test, y_sample, 'r-', alpha=0.1, linewidth=1)

ax.plot(X_test, X_test_full @ blr.m_n, 'r-', linewidth=2, label='Posterior mean')
ax.plot(X_test, X_test_full @ true_w, 'k--', linewidth=2, label='True function')
ax.set_xlabel('Factor Exposure', fontsize=11)
ax.set_ylabel('Return', fontsize=11)
ax.set_title('Posterior Sample Lines\n(Uncertainty in Weights)', fontsize=12, fontweight='bold')
ax.legend(fontsize=9)

# Plot 3: Posterior distribution of weights
ax = axes[2]

# Create grid for 2D posterior visualization
w0_range = np.linspace(blr.m_n[0] - 3*np.sqrt(blr.S_n[0,0]), 
                        blr.m_n[0] + 3*np.sqrt(blr.S_n[0,0]), 100)
w1_range = np.linspace(blr.m_n[1] - 3*np.sqrt(blr.S_n[1,1]), 
                        blr.m_n[1] + 3*np.sqrt(blr.S_n[1,1]), 100)
W0, W1 = np.meshgrid(w0_range, w1_range)
pos = np.dstack((W0, W1))

rv = stats.multivariate_normal(blr.m_n, blr.S_n)
Z = rv.pdf(pos)

ax.contourf(W0, W1, Z, levels=20, cmap='Reds', alpha=0.8)
ax.contour(W0, W1, Z, levels=5, colors='darkred', linewidths=0.5)
ax.scatter([true_w[0]], [true_w[1]], c='blue', s=100, marker='*', 
           label=f'True: ({true_w[0]:.2f}, {true_w[1]:.2f})', zorder=5)
ax.scatter([blr.m_n[0]], [blr.m_n[1]], c='red', s=100, marker='x', 
           linewidths=3, label=f'MAP: ({blr.m_n[0]:.2f}, {blr.m_n[1]:.2f})', zorder=5)
ax.scatter([ols.coef_[0]], [ols.coef_[1]], c='green', s=100, marker='+', 
           linewidths=3, label=f'OLS: ({ols.coef_[0]:.2f}, {ols.coef_[1]:.2f})', zorder=5)
ax.set_xlabel('w₀ (Intercept)', fontsize=11)
ax.set_ylabel('w₁ (Slope)', fontsize=11)
ax.set_title('Posterior Distribution of Weights', fontsize=12, fontweight='bold')
ax.legend(fontsize=9)

plt.tight_layout()
plt.show()

# Comparison table
print("\n" + "=" * 60)
print("📚 OLS VS BAYESIAN LINEAR REGRESSION")
print("=" * 60)
print("""
┌─────────────────────┬─────────────────────────┬─────────────────────────┐
│ Aspect              │ OLS                     │ Bayesian LR             │
├─────────────────────┼─────────────────────────┼─────────────────────────┤
│ Weights             │ Point estimates         │ Probability distribution│
│ Uncertainty         │ SE via residuals        │ Full posterior          │
│ Predictions         │ Point predictions       │ Predictive distribution │
│ Regularization      │ None (add manually)     │ Built-in via prior      │
│ Small data          │ Overfits                │ Prior regularizes       │
│ Out-of-sample       │ No uncertainty growth   │ Uncertainty increases   │
│ Computation         │ Closed form             │ Closed form (conjugate) │
└─────────────────────┴─────────────────────────┴─────────────────────────┘
""")

print(f"\n💡 Interview Answer Key Points:")
print("   1. Bayesian LR gives distribution over weights, not just point estimates")
print("   2. Predictive uncertainty grows away from training data")
print("   3. Prior precision α acts like L2 regularization strength")
print("   4. Enables principled uncertainty quantification for risk management")
print("   5. Finance use: factor models, return prediction with confidence bands")

---

## Question 8: Naive Bayes Classifier

### 📋 Interview Question:
*"Explain the Naive Bayes classifier. What is the 'naive' assumption, and when does it work well?"*

### 🎯 Key Concepts:
- **Naive Assumption:** Features are conditionally independent given class
- **Formula:** $P(y|x_1,...,x_n) \propto P(y) \prod_i P(x_i|y)$
- **Variants:** Gaussian, Multinomial, Bernoulli NB

In [ ]:
print("=" * 60)
print("NAIVE BAYES: From Scratch Implementation")
print("=" * 60)

class GaussianNaiveBayes:
    """
    Gaussian Naive Bayes classifier from scratch.
    
    Assumes: P(x_i | y) ~ Normal(μ_yi, σ_yi²)
    
    Classification: argmax_y P(y) * ∏_i P(x_i | y)
    """
    
    def __init__(self):
        self.classes = None
        self.class_priors = {}
        self.means = {}
        self.stds = {}
        
    def fit(self, X, y):
        """Estimate class priors and class-conditional feature distributions."""
        self.classes = np.unique(y)
        n_samples = len(y)
        
        for c in self.classes:
            X_c = X[y == c]
            
            # Class prior: P(y=c)
            self.class_priors[c] = len(X_c) / n_samples
            
            # Class-conditional means and stds: P(x_i | y=c) ~ N(μ_ci, σ_ci)
            self.means[c] = np.mean(X_c, axis=0)
            self.stds[c] = np.std(X_c, axis=0) + 1e-9  # Add small constant for stability
            
        return self
    
    def _log_likelihood(self, X, c):
        """Calculate log P(X | y=c) assuming independence."""
        # Log of product = sum of logs
        # log P(x | y=c) = Σ_i log N(x_i | μ_ci, σ_ci)
        log_probs = -0.5 * np.sum(((X - self.means[c]) / self.stds[c])**2, axis=1)
        log_probs -= np.sum(np.log(self.stds[c]))
        return log_probs
    
    def predict_proba(self, X):
        """Calculate posterior probabilities for each class."""
        log_posteriors = []
        
        for c in self.classes:
            log_prior = np.log(self.class_priors[c])
            log_likelihood = self._log_likelihood(X, c)
            log_posteriors.append(log_prior + log_likelihood)
        
        log_posteriors = np.array(log_posteriors).T
        
        # Normalize (softmax)
        max_log = np.max(log_posteriors, axis=1, keepdims=True)
        log_posteriors -= max_log
        posteriors = np.exp(log_posteriors)
        posteriors /= np.sum(posteriors, axis=1, keepdims=True)
        
        return posteriors
    
    def predict(self, X):
        """Predict class labels."""
        posteriors = self.predict_proba(X)
        return self.classes[np.argmax(posteriors, axis=1)]


# Application: Market Regime Classification
np.random.seed(42)

# Generate synthetic market data
# Features: momentum, volatility, volume
n_per_class = 200

# Class 0: Bear market (low momentum, high vol)
X_bear = np.random.multivariate_normal(
    mean=[-0.5, 0.3, -0.2],
    cov=[[0.1, 0.02, 0], [0.02, 0.1, 0], [0, 0, 0.1]],
    size=n_per_class
)

# Class 1: Bull market (high momentum, low vol)
X_bull = np.random.multivariate_normal(
    mean=[0.5, -0.2, 0.3],
    cov=[[0.1, -0.02, 0], [-0.02, 0.1, 0], [0, 0, 0.1]],
    size=n_per_class
)

# Class 2: Sideways (neutral)
X_sideways = np.random.multivariate_normal(
    mean=[0, 0, 0],
    cov=[[0.08, 0, 0], [0, 0.08, 0], [0, 0, 0.08]],
    size=n_per_class
)

X = np.vstack([X_bear, X_bull, X_sideways])
y = np.array([0]*n_per_class + [1]*n_per_class + [2]*n_per_class)
feature_names = ['Momentum', 'Volatility', 'Volume']
class_names = ['Bear', 'Bull', 'Sideways']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"\n📊 Market Regime Classification Problem")
print(f"   Features: {feature_names}")
print(f"   Classes: {class_names}")
print(f"   Train samples: {len(X_train)}, Test samples: {len(X_test)}")

# Fit models
# 1. Our implementation
gnb_scratch = GaussianNaiveBayes()
gnb_scratch.fit(X_train, y_train)
y_pred_scratch = gnb_scratch.predict(X_test)

# 2. Sklearn implementation
gnb_sklearn = GaussianNB()
gnb_sklearn.fit(X_train, y_train)
y_pred_sklearn = gnb_sklearn.predict(X_test)

print(f"\n📈 Results:")
print(f"   Our implementation accuracy: {accuracy_score(y_test, y_pred_scratch):.2%}")
print(f"   Sklearn accuracy: {accuracy_score(y_test, y_pred_sklearn):.2%}")

# Show learned parameters
print(f"\n📐 Learned Parameters (Class-Conditional Distributions):")
for i, c in enumerate(gnb_scratch.classes):
    print(f"\n   Class {c} ({class_names[c]}):")
    print(f"      Prior P(y={c}): {gnb_scratch.class_priors[c]:.3f}")
    print(f"      Feature means: {gnb_scratch.means[c]}")
    print(f"      Feature stds:  {gnb_scratch.stds[c]}")

# Visualization
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Plot 1: Feature distributions by class
ax = axes[0]
for i, c in enumerate(gnb_scratch.classes):
    X_c = X_train[y_train == c]
    for j, feat in enumerate(feature_names):
        x_range = np.linspace(X_c[:, j].min() - 0.5, X_c[:, j].max() + 0.5, 100)
        y_pdf = stats.norm.pdf(x_range, gnb_scratch.means[c][j], gnb_scratch.stds[c][j])
        ax.plot(x_range, y_pdf + j*2, label=f'{class_names[c]}-{feat}' if i==0 else '', 
                alpha=0.7, linewidth=2)

ax.set_xlabel('Feature Value', fontsize=11)
ax.set_ylabel('Density (stacked)', fontsize=11)
ax.set_title('Class-Conditional Feature Distributions', fontsize=12, fontweight='bold')
ax.legend(fontsize=8)

# Plot 2: Decision boundaries (2D projection)
ax = axes[1]
h = 0.05
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

# Use mean of third feature for prediction
X_grid = np.c_[xx.ravel(), yy.ravel(), np.full(xx.ravel().shape, X[:, 2].mean())]
Z = gnb_scratch.predict(X_grid)
Z = Z.reshape(xx.shape)

ax.contourf(xx, yy, Z, alpha=0.3, cmap='RdYlBu')
colors = ['red', 'blue', 'gray']
for i, c in enumerate(gnb_scratch.classes):
    ax.scatter(X[y==c, 0], X[y==c, 1], c=colors[i], label=class_names[c], alpha=0.6, s=30)

ax.set_xlabel(f'{feature_names[0]}', fontsize=11)
ax.set_ylabel(f'{feature_names[1]}', fontsize=11)
ax.set_title('Decision Boundaries\n(Momentum vs Volatility)', fontsize=12, fontweight='bold')
ax.legend()

# Plot 3: Posterior probabilities for test samples
ax = axes[2]
proba = gnb_scratch.predict_proba(X_test[:30])
x_pos = np.arange(30)
width = 0.25

for i, c in enumerate(gnb_scratch.classes):
    ax.bar(x_pos + i*width, proba[:, i], width, label=class_names[c], alpha=0.8)

ax.set_xlabel('Test Sample', fontsize=11)
ax.set_ylabel('Posterior Probability', fontsize=11)
ax.set_title('Posterior Probabilities\n(First 30 Test Samples)', fontsize=12, fontweight='bold')
ax.legend()
ax.set_xticks(x_pos[::5] + width)
ax.set_xticklabels(x_pos[::5])

plt.tight_layout()
plt.show()

# Explain the naive assumption
print("\n" + "=" * 60)
print("📚 THE NAIVE ASSUMPTION")
print("=" * 60)
print("""
Full Bayes:     P(y|x₁,x₂,...,xₙ) ∝ P(y) × P(x₁,x₂,...,xₙ|y)

Naive Bayes:    P(y|x₁,x₂,...,xₙ) ∝ P(y) × ∏ᵢ P(xᵢ|y)

The "naive" assumption: Features are conditionally independent given class.
This means: P(x₁,x₂|y) = P(x₁|y) × P(x₂|y)

WHY IT WORKS DESPITE BEING "WRONG":
1. Classification only needs correct ranking, not exact probabilities
2. Feature dependencies often don't change which class has highest posterior
3. High-dimensional problems benefit from reduced parameter count
4. Works well when class-conditional correlations are similar across classes
""")

print(f"\n💡 Interview Answer Key Points:")
print("   1. Naive = conditional independence assumption P(X|y) = ∏P(xᵢ|y)")
print("   2. Reduces parameters from O(d²) to O(d) per class")
print("   3. Works well for high-dimensional data (text, spam filtering)")
print("   4. Fast training and prediction - good for real-time classification")
print("   5. Finance: sentiment classification, regime detection, alert systems")

---

## Question 9: Credible Intervals vs Confidence Intervals

### 📋 Interview Question:
*"Explain the difference between credible intervals and confidence intervals. Why does the distinction matter in practice?"*

### 🎯 Key Concepts:
- **Confidence Interval:** Frequentist - covers true parameter in X% of repeated experiments
- **Credible Interval:** Bayesian - X% probability that parameter lies in interval
- **Interpretation:** Direct probability vs long-run frequency

In [ ]:
print("=" * 60)
print("CREDIBLE INTERVALS VS CONFIDENCE INTERVALS")
print("=" * 60)

# Scenario: Estimate Sharpe ratio of a strategy
np.random.seed(42)

# True parameters
true_sharpe = 1.2
true_mu = 0.15  # 15% annual return
true_sigma = true_mu / true_sharpe  # Volatility to achieve target Sharpe

# Generate sample returns (monthly)
n_months = 36  # 3 years of data
monthly_mu = true_mu / 12
monthly_sigma = true_sigma / np.sqrt(12)
returns = np.random.normal(monthly_mu, monthly_sigma, n_months)

# Sample statistics
sample_mu = np.mean(returns) * 12  # Annualized
sample_sigma = np.std(returns, ddof=1) * np.sqrt(12)
sample_sharpe = sample_mu / sample_sigma

print(f"\n📊 Scenario: Estimate Strategy Sharpe Ratio")
print(f"   True Sharpe: {true_sharpe:.2f}")
print(f"   Sample Sharpe: {sample_sharpe:.2f}")
print(f"   Sample size: {n_months} months")

# ============== FREQUENTIST: Confidence Interval ==============
print("\n" + "=" * 50)
print("🔵 FREQUENTIST: 95% Confidence Interval")
print("=" * 50)

# Sharpe ratio SE (Lo, 2002 approximation)
# SE(SR) ≈ sqrt((1 + SR²/2) / n)
se_sharpe = np.sqrt((1 + sample_sharpe**2 / 2) / n_months)

# Confidence interval using t-distribution
t_crit = stats.t.ppf(0.975, df=n_months-1)
ci_freq = (sample_sharpe - t_crit * se_sharpe, sample_sharpe + t_crit * se_sharpe)

print(f"   Sample Sharpe: {sample_sharpe:.3f}")
print(f"   Standard Error: {se_sharpe:.3f}")
print(f"   95% CI: [{ci_freq[0]:.3f}, {ci_freq[1]:.3f}]")
print(f"\n   ⚠️ INTERPRETATION (Frequentist):")
print(f"   'If we repeated this experiment many times,")
print(f"    95% of calculated CIs would contain the true Sharpe.'")
print(f"   We CANNOT say: 'There's 95% probability true Sharpe is in this interval.'")

# ============== BAYESIAN: Credible Interval ==============
print("\n" + "=" * 50)
print("🟢 BAYESIAN: 95% Credible Interval")
print("=" * 50)

# Use MCMC to get posterior of Sharpe ratio
def sharpe_posterior_samples(returns, n_samples=10000):
    """Sample from posterior of Sharpe ratio using bootstrap-like approach."""
    # Simple approach: use conjugate Normal-InverseGamma posterior
    # Then compute Sharpe = mu / sigma for each posterior sample
    
    n = len(returns)
    x_bar = np.mean(returns)
    s2 = np.var(returns, ddof=1)
    
    # Prior: non-informative
    # Posterior for sigma^2: Inv-Chi-squared(n-1, s^2)
    # Posterior for mu | sigma^2: Normal(x_bar, sigma^2/n)
    
    sigma2_samples = (n-1) * s2 / np.random.chisquare(n-1, n_samples)
    sigma_samples = np.sqrt(sigma2_samples)
    mu_samples = np.random.normal(x_bar, sigma_samples / np.sqrt(n))
    
    # Annualize
    mu_annual = mu_samples * 12
    sigma_annual = sigma_samples * np.sqrt(12)
    sharpe_samples = mu_annual / sigma_annual
    
    return sharpe_samples

sharpe_samples = sharpe_posterior_samples(returns, n_samples=50000)

# Credible intervals
hdi_95 = np.percentile(sharpe_samples, [2.5, 97.5])  # Equal-tailed
posterior_mean = np.mean(sharpe_samples)
posterior_median = np.median(sharpe_samples)

print(f"   Posterior Mean: {posterior_mean:.3f}")
print(f"   Posterior Median: {posterior_median:.3f}")
print(f"   95% Equal-Tailed CI: [{hdi_95[0]:.3f}, {hdi_95[1]:.3f}]")
print(f"\n   ✅ INTERPRETATION (Bayesian):")
print(f"   'Given the data and prior, there is 95% probability")
print(f"    that the true Sharpe ratio lies in this interval.'")

# Additional Bayesian insights
prob_positive = np.mean(sharpe_samples > 0)
prob_above_1 = np.mean(sharpe_samples > 1.0)
prob_above_2 = np.mean(sharpe_samples > 2.0)

print(f"\n   📈 Probabilistic Statements:")
print(f"   P(Sharpe > 0) = {prob_positive:.1%}")
print(f"   P(Sharpe > 1.0) = {prob_above_1:.1%}")
print(f"   P(Sharpe > 2.0) = {prob_above_2:.1%}")

# Visualization
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Plot 1: Confidence Interval interpretation (simulation)
ax = axes[0]
np.random.seed(123)
n_experiments = 50
cis_list = []
contains_true = []

for _ in range(n_experiments):
    sample = np.random.normal(monthly_mu, monthly_sigma, n_months)
    sr = np.mean(sample) * 12 / (np.std(sample, ddof=1) * np.sqrt(12))
    se = np.sqrt((1 + sr**2 / 2) / n_months)
    ci = (sr - t_crit * se, sr + t_crit * se)
    cis_list.append((sr, ci))
    contains_true.append(ci[0] <= true_sharpe <= ci[1])

for i, (sr, ci) in enumerate(cis_list):
    color = 'green' if contains_true[i] else 'red'
    ax.plot([ci[0], ci[1]], [i, i], color=color, linewidth=1.5)
    ax.scatter([sr], [i], color=color, s=20, zorder=5)

ax.axvline(true_sharpe, color='blue', linestyle='--', linewidth=2, 
           label=f'True SR = {true_sharpe:.1f}')
coverage = sum(contains_true) / n_experiments
ax.set_xlabel('Sharpe Ratio', fontsize=11)
ax.set_ylabel('Experiment', fontsize=11)
ax.set_title(f'Frequentist CI Coverage\n{sum(contains_true)}/{n_experiments} = {coverage:.0%} contain true value', 
             fontsize=12, fontweight='bold')
ax.legend()

# Plot 2: Posterior distribution with credible interval
ax = axes[1]
ax.hist(sharpe_samples, bins=100, density=True, alpha=0.7, color='green', edgecolor='none')
ax.axvline(true_sharpe, color='red', linestyle='--', linewidth=2, 
           label=f'True SR = {true_sharpe:.2f}')
ax.axvline(posterior_mean, color='green', linestyle=':', linewidth=2,
           label=f'Posterior mean = {posterior_mean:.2f}')
ax.axvspan(hdi_95[0], hdi_95[1], alpha=0.2, color='blue', 
           label=f'95% Credible Interval')

# Add vertical lines at CI bounds
ax.axvline(hdi_95[0], color='blue', linestyle='-', linewidth=1.5, alpha=0.7)
ax.axvline(hdi_95[1], color='blue', linestyle='-', linewidth=1.5, alpha=0.7)

ax.set_xlabel('Sharpe Ratio', fontsize=11)
ax.set_ylabel('Posterior Density', fontsize=11)
ax.set_title('Bayesian Posterior Distribution\nwith 95% Credible Interval', fontsize=12, fontweight='bold')
ax.legend(fontsize=9)

# Plot 3: Comparison of intervals
ax = axes[2]
methods = ['Frequentist\nConfidence', 'Bayesian\nCredible']
point_est = [sample_sharpe, posterior_mean]
lower = [ci_freq[0], hdi_95[0]]
upper = [ci_freq[1], hdi_95[1]]
colors = ['blue', 'green']

x_pos = np.arange(2)
for i in range(2):
    ax.errorbar(x_pos[i], point_est[i], 
                yerr=[[point_est[i] - lower[i]], [upper[i] - point_est[i]]],
                fmt='o', markersize=10, color=colors[i], capsize=10, capthick=2,
                elinewidth=2)

ax.axhline(true_sharpe, color='red', linestyle='--', linewidth=2, 
           label=f'True Sharpe = {true_sharpe:.1f}')
ax.set_xticks(x_pos)
ax.set_xticklabels(methods, fontsize=11)
ax.set_ylabel('Sharpe Ratio', fontsize=11)
ax.set_title('Interval Comparison', fontsize=12, fontweight='bold')
ax.legend()

# Add interval width annotations
for i in range(2):
    width = upper[i] - lower[i]
    ax.annotate(f'Width: {width:.2f}', xy=(x_pos[i], lower[i] - 0.1), 
                ha='center', fontsize=10, color=colors[i])

plt.tight_layout()
plt.show()

# Summary table
print("\n" + "=" * 60)
print("📚 CREDIBLE VS CONFIDENCE INTERVALS")
print("=" * 60)
print("""
┌──────────────────┬────────────────────────────┬────────────────────────────┐
│ Aspect           │ Confidence Interval        │ Credible Interval          │
├──────────────────┼────────────────────────────┼────────────────────────────┤
│ Framework        │ Frequentist                │ Bayesian                   │
│ Parameter        │ Fixed, unknown             │ Random variable            │
│ Interpretation   │ Long-run coverage          │ Direct probability         │
│ Can say          │ "95% of CIs cover θ"       │ "95% prob θ in interval"   │
│ Cannot say       │ "95% prob θ is here"       │ N/A                        │
│ Requires         │ Sampling distribution      │ Prior + Posterior          │
│ Width depends on │ Sample size, variance      │ Sample + Prior precision   │
└──────────────────┴────────────────────────────┴────────────────────────────┘
""")

print(f"\n💡 Interview Answer Key Points:")
print("   1. Confidence: procedure property; Credible: probability statement")
print("   2. CI: 'If repeated, 95% of intervals contain true value'")
print("   3. Credible: 'Given data, 95% probability parameter is in interval'")
print("   4. Bayesian allows direct probability questions: P(θ > threshold)")
print("   5. For risk management, credible intervals are more intuitive")

---

## Question 10: Bayesian Model Selection

### 📋 Interview Question:
*"How do you compare models in a Bayesian framework? Explain Bayes factors and information criteria."*

### 🎯 Key Concepts:
- **Bayes Factor:** Ratio of marginal likelihoods $BF_{12} = P(D|M_1) / P(D|M_2)$
- **Marginal Likelihood:** Evidence = $\int P(D|\theta, M) P(\theta|M) d\theta$
- **Information Criteria:** BIC approximates log Bayes factor

In [ ]:
print("=" * 60)
print("BAYESIAN MODEL SELECTION: Bayes Factors & Information Criteria")
print("=" * 60)

# Generate data from a quadratic relationship
np.random.seed(42)
n_points = 50
x = np.linspace(-3, 3, n_points)
noise_std = 1.5

# True model: y = 1 + 0.5x + 0.3x²
true_a, true_b, true_c = 1.0, 0.5, 0.3
y_true = true_a + true_b * x + true_c * x**2
y = y_true + np.random.randn(n_points) * noise_std

print(f"\n📊 Problem: Which polynomial degree fits best?")
print(f"   True model: y = {true_a} + {true_b}x + {true_c}x²")
print(f"   Testing models: Linear (deg=1), Quadratic (deg=2), Cubic (deg=3)")

def fit_polynomial(x, y, degree):
    """Fit polynomial and return metrics."""
    X = np.vstack([x**i for i in range(degree + 1)]).T
    
    # OLS fit
    coeffs = np.linalg.lstsq(X, y, rcond=None)[0]
    y_pred = X @ coeffs
    residuals = y - y_pred
    
    # Metrics
    n = len(y)
    k = degree + 1  # Number of parameters
    rss = np.sum(residuals**2)
    sigma2_mle = rss / n
    
    # Log-likelihood
    log_likelihood = -n/2 * np.log(2 * np.pi * sigma2_mle) - rss / (2 * sigma2_mle)
    
    # Information criteria
    aic = 2 * k - 2 * log_likelihood
    bic = k * np.log(n) - 2 * log_likelihood
    
    # Adjusted R²
    tss = np.sum((y - np.mean(y))**2)
    r2 = 1 - rss / tss
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - k - 1)
    
    return {
        'coeffs': coeffs,
        'y_pred': y_pred,
        'rss': rss,
        'log_likelihood': log_likelihood,
        'aic': aic,
        'bic': bic,
        'r2': r2,
        'adj_r2': adj_r2,
        'n_params': k
    }


# Fit models of different degrees
degrees = [1, 2, 3, 4, 5]
results = {d: fit_polynomial(x, y, d) for d in degrees}

print("\n📈 Model Comparison Results:")
print("-" * 80)
print(f"{'Degree':<10} {'Params':<10} {'R²':<10} {'Adj R²':<10} {'AIC':<12} {'BIC':<12}")
print("-" * 80)
for d in degrees:
    r = results[d]
    print(f"{d:<10} {r['n_params']:<10} {r['r2']:.4f}    {r['adj_r2']:.4f}    {r['aic']:.2f}      {r['bic']:.2f}")

# Find best model by each criterion
best_aic = min(degrees, key=lambda d: results[d]['aic'])
best_bic = min(degrees, key=lambda d: results[d]['bic'])

print(f"\n✅ Best Model Selection:")
print(f"   By AIC: Degree {best_aic}")
print(f"   By BIC: Degree {best_bic}")

# Bayes Factor approximation via BIC
print("\n" + "=" * 50)
print("BAYES FACTORS (Approximated via BIC)")
print("=" * 50)

reference_model = 2  # Compare against quadratic
print(f"\nComparing models against Model {reference_model} (Quadratic):")
print("-" * 60)

bic_ref = results[reference_model]['bic']
for d in degrees:
    if d != reference_model:
        delta_bic = results[d]['bic'] - bic_ref
        # BF ≈ exp(-0.5 * ΔBIC)
        log_bf = -0.5 * delta_bic
        
        # Interpretation
        if delta_bic > 10:
            evidence = "Very strong against"
        elif delta_bic > 6:
            evidence = "Strong against"
        elif delta_bic > 2:
            evidence = "Positive against"
        elif delta_bic > -2:
            evidence = "Weak/inconclusive"
        elif delta_bic > -6:
            evidence = "Positive for"
        elif delta_bic > -10:
            evidence = "Strong for"
        else:
            evidence = "Very strong for"
            
        print(f"   Model {d} vs Model {reference_model}: ΔBIC = {delta_bic:.2f}, log(BF) ≈ {log_bf:.2f}")
        print(f"      → {evidence} model {d}")

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Data and fitted models
ax = axes[0, 0]
ax.scatter(x, y, c='black', alpha=0.6, s=40, label='Data')
ax.plot(x, y_true, 'k--', linewidth=2, label='True function')

colors = ['blue', 'green', 'red', 'orange', 'purple']
x_smooth = np.linspace(x.min(), x.max(), 200)
for d, color in zip([1, 2, 3], colors[:3]):
    X_smooth = np.vstack([x_smooth**i for i in range(d + 1)]).T
    y_smooth = X_smooth @ results[d]['coeffs']
    ax.plot(x_smooth, y_smooth, color=color, linewidth=2, label=f'Degree {d}')

ax.set_xlabel('x', fontsize=11)
ax.set_ylabel('y', fontsize=11)
ax.set_title('Polynomial Fits', fontsize=12, fontweight='bold')
ax.legend()

# Plot 2: Information criteria comparison
ax = axes[0, 1]
x_pos = np.arange(len(degrees))
width = 0.35

aic_values = [results[d]['aic'] for d in degrees]
bic_values = [results[d]['bic'] for d in degrees]

bars1 = ax.bar(x_pos - width/2, aic_values, width, label='AIC', color='blue', alpha=0.7)
bars2 = ax.bar(x_pos + width/2, bic_values, width, label='BIC', color='green', alpha=0.7)

# Highlight best models
ax.bar(best_aic - 1 - width/2, aic_values[best_aic-1], width, color='blue', edgecolor='red', linewidth=3)
ax.bar(best_bic - 1 + width/2, bic_values[best_bic-1], width, color='green', edgecolor='red', linewidth=3)

ax.set_xlabel('Polynomial Degree', fontsize=11)
ax.set_ylabel('Information Criterion', fontsize=11)
ax.set_title('AIC vs BIC Model Selection', fontsize=12, fontweight='bold')
ax.set_xticks(x_pos)
ax.set_xticklabels(degrees)
ax.legend()

# Plot 3: Bayes Factor interpretation scale
ax = axes[1, 0]
categories = ['ΔBIC < -10', '-10 to -6', '-6 to -2', '-2 to 2', '2 to 6', '6 to 10', 'ΔBIC > 10']
interpretations = ['Very Strong\nFor', 'Strong\nFor', 'Positive\nFor', 
                   'Weak/\nInconclusive', 'Positive\nAgainst', 'Strong\nAgainst', 'Very Strong\nAgainst']
colors_scale = ['darkgreen', 'green', 'lightgreen', 'gray', 'lightsalmon', 'red', 'darkred']

bars = ax.barh(range(len(categories)), [1]*len(categories), color=colors_scale, edgecolor='black')
ax.set_yticks(range(len(categories)))
ax.set_yticklabels([f'{cat}\n{interp}' for cat, interp in zip(categories, interpretations)], fontsize=9)
ax.set_xlabel('Scale', fontsize=11)
ax.set_title('Bayes Factor Evidence Scale\n(Kass & Raftery Guidelines)', fontsize=12, fontweight='bold')
ax.set_xlim(0, 1.2)
ax.set_xticks([])

# Plot 4: Out-of-sample prediction
ax = axes[1, 1]

# Generate test data
x_test = np.linspace(-4, 4, 100)
y_test_true = true_a + true_b * x_test + true_c * x_test**2

for d, color in zip([1, 2, 5], ['blue', 'green', 'purple']):
    X_test = np.vstack([x_test**i for i in range(d + 1)]).T
    y_test_pred = X_test @ results[d]['coeffs']
    ax.plot(x_test, y_test_pred, color=color, linewidth=2, 
            label=f'Degree {d} (BIC={results[d]["bic"]:.1f})')

ax.plot(x_test, y_test_true, 'k--', linewidth=2, label='True function')
ax.scatter(x, y, c='black', alpha=0.4, s=30)
ax.axvspan(x.min(), x.max(), alpha=0.1, color='gray', label='Training range')

ax.set_xlabel('x', fontsize=11)
ax.set_ylabel('y', fontsize=11)
ax.set_title('Extrapolation: Why Model Selection Matters', fontsize=12, fontweight='bold')
ax.legend(fontsize=9)
ax.set_ylim(-20, 30)

plt.tight_layout()
plt.show()

# Summary
print("\n" + "=" * 60)
print("📚 BAYESIAN MODEL SELECTION METHODS")
print("=" * 60)
print("""
1. BAYES FACTOR (BF):
   - BF₁₂ = P(D|M₁) / P(D|M₂) = Evidence ratio
   - BF > 1: Favor M₁; BF < 1: Favor M₂
   - Interpretation: Kass & Raftery scale (2log(BF))

2. MARGINAL LIKELIHOOD (Evidence):
   - P(D|M) = ∫ P(D|θ,M) P(θ|M) dθ
   - Automatically penalizes complexity
   - Hard to compute for complex models

3. INFORMATION CRITERIA:
   - AIC = 2k - 2log(L) → Prediction focused
   - BIC = klog(n) - 2log(L) → Model selection, approximates log(BF)
   - BIC penalizes complexity more heavily for large n

4. CROSS-VALIDATION (Bayesian LOO):
   - WAIC (Widely Applicable IC)
   - LOO-CV via Pareto Smoothed Importance Sampling
""")

print(f"\n💡 Interview Answer Key Points:")
print("   1. Bayes factor = ratio of marginal likelihoods")
print("   2. Naturally penalizes complexity via Occam's razor")
print("   3. BIC ≈ -2 × log(Bayes Factor) for large n")
print("   4. AIC: prediction; BIC: model selection (more conservative)")
print("   5. Finance: compare factor models, regime-switching specifications")